In [1]:
# # I am practicing pushing to GitHub

# practice = "I am testing this out"

# print(practice)

In [2]:
# Dependencies and Setup
import pandas as pd

In [3]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [4]:
# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [5]:
#examine the data
# school_data.head()

In [6]:
#examine the data
# student_data.head()

In [7]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [8]:
# assign summary data to variables
schools_total = school_data_complete["School ID"].nunique()
student_total = school_data_complete["Student ID"].nunique()
total_budget = school_data_complete["budget"].unique().sum()
average_math = school_data_complete["math_score"].mean(axis=0)
average_reading = school_data_complete["reading_score"].mean(axis=0)
passing_math_count = school_data_complete.math_score[(school_data_complete.math_score  >= 70)].count()
passing_math_percent = passing_math_count / student_total * 100
passing_reading_count = school_data_complete.reading_score[(school_data_complete.reading_score >= 70)].count()
passing_reading_percent = passing_reading_count / student_total * 100 
passing_both_count = len(school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70) ])
passing_both_percent = (passing_both_count / student_total) * 100


In [9]:
# create dataframe
combined_df = pd.DataFrame({"Total Schools": [schools_total],
                            "Total Students": [student_total],
                            "Total Budget":   [total_budget], # is this value correct?
                            "Average Math Score": [f'{round(average_math,1)}%'],
                            "Average Reading Score": [f'{round(average_reading, 1)}%'],
                            "Students Passing Math": [f'{round(passing_math_percent, 1)}%'],
                            "Students Passing Reading": [f'{round(passing_reading_percent, 1)}%'],
                            "Overall Passing": [f'{round(passing_both_percent, 1)}%']
                           
                           })

# trying to figure out the formatting for the budget
combined_df['Total Budget'] = combined_df['Total Budget'].astype(int).apply(lambda x: "${:,}".format(x))
combined_df['Total Students'] = combined_df['Total Students'].astype(int).apply(lambda x: "{:,}".format(x))
combined_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Overall Passing
0,15,"39,170","$24,649,428",79.0%,81.9%,75.0%,85.8%,65.2%


In [32]:
# # create lists for Group By Schools df
# schools = school_data_complete["school_name"].unique()
# school_types = school_data_complete["school_name"].loc[schools, school_data_complete["type"]]
# school_type



IndexingError: Too many indexers

In [17]:
# # Group by schools
# groupby_school = school_data_complete.groupby(['school_name']).count()
# groupby_school = groupby_school[["type","size"]]
# groupby_school = groupby_school.rename(columns={"type":"School Type","size":"Total Students"})
# groupby_school



,School Type,Total Students
school_name,,
Bailey High School,4976,4976
Cabrera High School,1858,1858
Figueroa High School,2949,2949
Ford High School,2739,2739
Griffin High School,1468,1468
Hernandez High School,4635,4635
Holden High School,427,427
Huang High School,2917,2917
Johnson High School,4761,4761
